In [6]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
       
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 16 * 16, 128) 
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        
        x = F.relu(self.conv2(x))

        x = F.relu(self.conv3(x))
        
        x = self.pool(x)

        x = x.view(-1, 128 * 16 * 16)  

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# k折交叉验证
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset


transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

full_dataset = datasets.CIFAR10('./data/CIFAR10', train=True, download=True, transform=transforms_train)

def get_k_fold_data(k, i, dataset):
    fold_size = len(dataset) // k
    
    val_start = i * fold_size
    val_end = (i + 1) * fold_size if i < k - 1 else len(dataset)
    
    val_indices = list(range(val_start, val_end))
    train_indices = list(range(0, val_start)) + list(range(val_end, len(dataset)))

    return Subset(dataset, train_indices), Subset(dataset, val_indices)


def train(lr, train_loader):
    # 模型实例化和优化器设置
    model = CNN()
    optimizer = optim.Adamax(model.parameters(), lr=lr)

    # 设备设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    # 训练过程
    for epoch in range(1, 10):
        for batch_idx, (data, target) in enumerate(train_loader):
            # 将数据移动到设备上
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = torch.nn.CrossEntropyLoss()(output, target)
            loss.backward()
            optimizer.step()

    # 保存模型
    torch.save(model.state_dict(), f'./model/NULL_cnn.pt')

def valid(valid_loader):
    model = CNN()
    model.load_state_dict(torch.load(f'./model/NULL_cnn.pt'))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in valid_loader:
           
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(valid_loader.dataset)
    print(f'\nValid set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(valid_loader.dataset)} ({100. * correct / len(valid_loader.dataset):.0f}%)\n')
    return  correct / len(valid_loader.dataset)


k_folds = 5
for lr_ in range(1, 6):
    rate = 0.0
    lr = lr_ * 0.0005  #从0.0005到0.0025每隔0.0005进行一次k折交叉验证
    for i in range(k_folds):
        print(f"lr:{lr},FOLD: {i}")
        print("--------------------------------")

        train_subset, val_subset = get_k_fold_data(k_folds, i, full_dataset)

        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
        valid_loader = DataLoader(val_subset, batch_size=128, shuffle=False)
        train(lr, train_loader)
        rate += valid(valid_loader)
    print(f"学习率为：{lr}平均结果：{rate / k_folds}")


Files already downloaded and verified
lr:0.0005,FOLD: 0
--------------------------------

Valid set: Average loss: 0.8301, Accuracy: 7092/10000 (71%)

lr:0.0005,FOLD: 1
--------------------------------

Valid set: Average loss: 0.9110, Accuracy: 6841/10000 (68%)

lr:0.0005,FOLD: 2
--------------------------------

Valid set: Average loss: 0.8462, Accuracy: 7093/10000 (71%)

lr:0.0005,FOLD: 3
--------------------------------

Valid set: Average loss: 0.8906, Accuracy: 6932/10000 (69%)

lr:0.0005,FOLD: 4
--------------------------------

Valid set: Average loss: 0.8940, Accuracy: 6945/10000 (69%)

学习率为：0.0005平均结果：0.69806
lr:0.001,FOLD: 0
--------------------------------

Valid set: Average loss: 0.8045, Accuracy: 7259/10000 (73%)

lr:0.001,FOLD: 1
--------------------------------

Valid set: Average loss: 0.8622, Accuracy: 7189/10000 (72%)

lr:0.001,FOLD: 2
--------------------------------

Valid set: Average loss: 0.8085, Accuracy: 7284/10000 (73%)

lr:0.001,FOLD: 3
--------------------

In [7]:
# train
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 数据加载和预处理
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

train_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=True, download=True, transform=transforms_train),
    batch_size=64, shuffle=True)

def train():
    # 模型实例化和优化器设置
    model = CNN()
    optimizer = optim.Adamax(model.parameters(), lr=0.002)

    # 设备设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    # 训练过程
    for epoch in range(1, 10):
        for batch_idx, (data, target) in enumerate(train_loader):
            # 将数据移动到设备上
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = torch.nn.CrossEntropyLoss()(output, target)
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    # 保存模型
    torch.save(model.state_dict(), f'./model/NULL_cnn.pt')

train()

Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.297033
Train Epoch: 1 [640/50000 (1%)]	Loss: 2.267631
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.184319
Train Epoch: 1 [1920/50000 (4%)]	Loss: 2.154015
Train Epoch: 1 [2560/50000 (5%)]	Loss: 1.902373
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2.040782
Train Epoch: 1 [3840/50000 (8%)]	Loss: 1.942515
Train Epoch: 1 [4480/50000 (9%)]	Loss: 1.875883
Train Epoch: 1 [5120/50000 (10%)]	Loss: 1.870070
Train Epoch: 1 [5760/50000 (12%)]	Loss: 1.752329
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.787758
Train Epoch: 1 [7040/50000 (14%)]	Loss: 1.718637
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1.592201
Train Epoch: 1 [8320/50000 (17%)]	Loss: 1.631913
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1.726522
Train Epoch: 1 [9600/50000 (19%)]	Loss: 1.442886
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1.429687
Train Epoch: 1 [10880/50000 (22%)]	Loss: 1.299579
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1.605687
Train Epoch: 1 [12160/50000 (24%)]	Loss:

In [8]:
# test.py
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 数据加载
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.226, 0.224, 0.225))
])

test_loader = DataLoader(
    datasets.CIFAR10('./data/CIFAR10', train=False, download=True, transform=transforms_train),
    batch_size=1000, shuffle=True)

def test():
    model = CNN()
    model.load_state_dict(torch.load(f'./model/NULL_cnn.pt'))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')
test()


Files already downloaded and verified

Test set: Average loss: 0.9285, Accuracy: 7524/10000 (75%)

